# Analysis of Record for Calibration Data Aggregation Using Xvec
#### Xvec: Vector data cubes for Xarray

<div style="background-color: #f7f7f7; padding: 10px; border-radius: 5px;">
    <p><strong>Author:</strong> 
    <ul>
        <li>Irene Garousi-Nejad, <a href="mailto:igarousi@cuahsi.org">igarousi@cuahsi.org</a></li>
    </ul>
    </p>
    <p><strong>Last Modified:</strong> 1/24/25</p>
    <p><strong>Description:</strong> Vector data cubes, such as those enabled by <strong>GeoCube</strong> or <strong>Xvec</strong>, provide a powerful approach for performing spatial analysis of gridded water and climate-related datasets across spatial features like watersheds, administrative boundaries, and ecological regions. This notebook demonstrates an example of how xvec can be used to aggregate AORC data over several subbasins within the Great Salt Lake basin. The ability to directly interact with vector geometries while leveraging the computational efficiency of libraries like Dask enhances workflow scalability, making it feasible to analyze large, complex datasets. Both Xvec and GeoCube are Python libraries designed for spatial analysis, but they serve slightly different purposes. Xvec is generally recommended when working primarily with gridded datasets in xarray, especially for tasks requiring advanced vector operations or scalable zonal statistics. In contrast, GeoCube is ideal for workflows involving GeoPandas vector data, as it focuses on rasterizing vector data for integration with raster datasets. While GeoCube is not optimized for large-scale or parallel processing, xvec is scalable and leverages Dask to efficiently handle large datasets.</p>
    <p><strong>Software Requirements:</strong> This notebook has been tested using Python v3.11.8 using the following library versions:
    <blockquote>
        <ul>
            <li>cf_xarray==0.10.0</li>
            <li>dask==2024.4.1</li>
            <li>dask-geopandas==0.3.1</li>
            <li>fsspec==2024.3.1</li>
            <li>geopandas==0.14.3</li>
            <li>numpy==1.26.4</li>
            <li>pint-xarray==0.3</li>
            <li>rioxarray==0.15.3</li>
            <li>xarray==2024.3.0</li>
            <li>xarray-datatree==0.0.14</li>
            <li>xgcm==0.6.1</li>
            <li>xvec==0.3.1</li>
            <li>jupyter</li>
        </ul>
    </blockquote>
    </p>
    <p><strong>Supporting Files and Dependencies:</strong> 
        <ul>
            <li>Environment specification (requirements.txt)</li>
            <li>GSLSubbasins shapefile dataset (GSLSubbasins.*)</li>
        </ul>
    </p>
    <p><strong>References:</strong></p>
    <ul>
        <li><a href="https://r-spatial.org/r/2022/09/12/vdc.html">https://r-spatial.org/r/2022/09/12/vdc.html</a></li>
        <li><a href="https://xvec.readthedocs.io/en/latest/index.html">https://xvec.readthedocs.io/en/latest/index.html</a></li>
        <li><a href="https://corteva.github.io/geocube/stable/examples/zonal_statistics.html">https://corteva.github.io/geocube/stable/examples/zonal_statistics.html</a></li>
    </ul>
    
</div>

### 1. Prepare the Python Environment

Use the following command to ensure that all dependencies are installed in your environment. Note, these library versions have been pinned and tested for `Python 3.11.8`.

In [ ]:
!pip install -r requirements.txt

Import the libraries needed to run this notebook:

In [ ]:
import sys
import dask
import numpy
import xarray as xr
import fsspec
import xvec
import rioxarray
import geopandas as gpd
import matplotlib.pyplot as plt
from dask.distributed import Client

import warnings
warnings.filterwarnings("ignore")

We'll use `dask` to parallelize our code.

In [ ]:
# use a try accept loop so we only instantiate the client
# if it doesn't already exist.
try:
    print(client.dashboard_link)
except:    
    # The client should be customized to your workstation resources.
    # This is configured for a "Large" instance on ciroh.awi.2i2c.cloud
    # client = Client()
    client = Client(n_workers=12, memory_limit='10GB') 
    print(client.dashboard_link)

### 2. Access the AORC Precipitation Data

We'll be working with [AORC v1.1 precipitation data](https://noaa-nwm-retrospective-3-0-pds.s3.amazonaws.com/index.html#CONUS/zarr/forcing/precip.zarr/), which is used in the National Water Model version 3. These data are publicly available as part of the NOAA National Water Model v3.0 Retrospective archive on AWS registry of open data. These data are available in the Zarr format which offers a convienent and efficient means for slicing and subsetting very large datasets using libraries such as xarray. 

In [ ]:
bucket_url = 's3://noaa-nwm-retrospective-3-0-pds'
region = 'CONUS'
variable = 'precip'

In [ ]:
# build a path to the zarr store that we want
s3path = f"{bucket_url}/{region}/zarr/forcing/{variable}.zarr"

# load these data using xarray
ds = xr.open_zarr(fsspec.get_mapper(s3path, anon=True), consolidated=True)

# preview the dataset
ds

This Dataset is indexed by `x` and `y` representing the spatial grid. When aggregating using `ds.xvec.zonal_stats`, we are replacing these two dimensions with a single one with `shapely` geometry.

Print the projection system of the dataset.

In [ ]:
ds.crs.grid_mapping_name 

### 3. Load the geometry data 

Use `Geopandas` to load a watershed Shapefile that defines our area of interest and includes hydrological subbasins as a set of polygons.

In [ ]:
shp = gpd.read_file('./GSLSubbasins.shp')
shp.head()

Preview the polygons.

In [ ]:
shp.plot(facecolor='white', edgecolor='black')

Print the coordinate reference system of the shapefile.

In [ ]:
print(shp.crs)

<div style="border-left: 4px solid #007acc; background-color: #f9f9f9; padding: 10px; border-radius: 4px;">
<strong>Note:</strong> <code>Xvec</code> method does not require a rioxarray CRS attached to the object but requires a user to ensure that the data are using the same projection. 
</div>




### 4. Aligning Data Projections

We will reproject the shapefile to match the coordinate reference system (CRS) of the xarray dataset. To do this, we need to ensure that the CRS in the xarray dataset is properly set. Currently, the CRS is stored as a string in the metadata (`ds.crs.attrs['esri_pe_string']`) but is not correctly applied, meaning spatial operations like resampling and reprojection will not work. By using `rioxarray`’s `set_crs()` method, we can apply the CRS to the dataset in a standard geospatial format. The next cell extracts the CRS information from the metadata and then writes it to the dataset to ensure it is fully set and ready for spatial operations.

<div style="border-left: 4px solid #007acc; background-color: #f9f9f9; padding: 10px; border-radius: 4px;">
<strong>Note:</strong> This should only be run once. Running it a second time will cause an error because the CRS will no longer exist as metadata once it has been set and written to the dataset. 
</div>

In [ ]:
ds.rio.set_crs(ds.crs.attrs['esri_pe_string'])
ds.rio.write_crs(inplace=True);


Assign the CRS of the xarray dataset to the shapefile and ensure it has changed.

In [ ]:
shp = shp.to_crs(ds.rio.crs)
print(shp.crs)

### 5. Zonal Aggregation of AORC Over Sub-basins

We use zonal statistics, a common aggregation method that involves summarizing raster values within a geometry. `xvec.zonal_stats()` can be used for this purpose, as it preserves the structure and attributes of the original data cube (`ds`) while enabling indexing by polygons. The `xvec.zonal_stats()` function accepts a range of parameters, including both required and optional ones. The table below provides the description of these parameters.

| **Parameter**   | **Description**   | **Default**        | **Required**  |
|-----------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------|---------------|
| **`geometry`**   | A 1-D array-like object (e.g., `numpy.array` or `geopandas.GeoSeries`) containing shapely geometries such as Polygons or LineStrings.| - | Yes|
| **`x_coords`**   | Name of the coordinate containing `x` values (e.g., longitude or the first vertex coordinate of the geometry).| -  | Yes  |
| **`y_coords`**   | Name of the coordinate containing `y` values (e.g., latitude or the second vertex coordinate of the geometry).                                                                                                                                                 | -                  | Yes           |
| **`stats`**      | Spatial aggregation method. Options include standard methods like `"mean"`, `"min"`, `"max"`, `"quantile(q=0.2)"`, custom `Callable` functions, or sequences in `(name, func, {kwargs})` format.                                                               | `"mean"`           | No            |
| **`name`**       | Name of the dimension that will hold the geometry.                                                                                                                                                                                                            | `"geometry"`       | No            |
| **`index`**      | Determines whether the `GeoSeries` index is attached to the geometry dimension. Options: `True` (attach index), `False` (do not store index).                                                                                                                 | `None`             | No            |
| **`method`**     | Data extraction method: `"rasterize"` (faster, may lose detail), `"iterate"` (uses `geometry_mask`), `"exactextract"` (precise stats using raster cell coverage).                                                                                              | `"rasterize"`      | No            |
| **`all_touched`** | If `True`, considers all pixels touched by geometries. If `False`, includes only pixels whose centers fall within the geometry. Applies to `"rasterize"` and `"iterate"` methods.                                                                           | `False`            | No            |
| **`n_jobs`**     | Number of parallel threads for computation. Set to `-1` to use all available CPU cores. Applies only to the `"iterate"` method.                                                                                                                               | -                  | No            |


Try the `iterate` method, which allows us to specify the number of parallel threads for computation. Note that, as indicated in the parameters table, parallelization is only supported for the iterate method.

In [ ]:
%%time
aggregated_par = ds.xvec.zonal_stats(
    shp.geometry,      
    x_coords="x",       
    y_coords="y", 
    stats="mean",
    name="basin_geometry",    
    index=True,
    method="iterate",
    all_touched=True,
    n_jobs=-1
)

This original dataset is indexed by longitude and latitude representing the spatial grid. When aggregating using `ds.xvec.zonal_stats`, we are replacing these two dimensions with a single one with shapely geometry. Below is a preview of the dataset with the aligned vectors:

In [ ]:
aggregated_par

The current dataset includes vector data, allowing us to easily subset it for a specific subbasin using `aggregated_par.isel(geometry=0)`. However, we don’t know which subbasin this corresponds to. Adding basin names to the dataset makes it significantly easier to subset data based on basin names for various applications. This can be achieved using `assign_coords`, which requires: (1) a name for the new coordinate (`basin_name`), (2) the dimension to associate it with (`basin_geometry`), and (3) the values for the new coordinate (provided as `shp.name.values`).

In [ ]:
aggregated_par = (
    aggregated_par
    .assign_coords(basin_name=("basin_geometry", shp.name.values))
)

Preview the dataset with added basin names.

In [ ]:
aggregated_par

<div style="background-color: #e0f7fa; padding: 10px; border-radius: 5px;">
    <strong>Subset Example:</strong> Plot the spatially averaged precipitation from the AORC data for January 2019 in the Jordan River subbasin within the Great Salt Lake Basin. Note that the area of this subbasin is approximately 9,258 square kilometers (calculated using <code>shp.geometry.area / 1e6</code>), and since the precipitation data are at an hourly time step, this task may take some time to complete.
</div>



In [ ]:
%%time

dat = (aggregated_par
       .where(aggregated_par.basin_name == "Jordan", drop=True)
       .sel(time=slice('2019-01-01', '2019-02-01'))
       .RAINRATE.compute() # triggers the computation
)

In [ ]:
fig, ax = plt.subplots()

dat.plot(ax=ax)
ax.set_title(f'Spatially Averaged Precipitation for Jordan River Subbasin')
ax.set_xlabel('Date')
plt.show()

### 6. Other aggregation examples

We can pass a list of aggregation methods, which will create an additional dimension in the resulting dataset. This can be done by using:

> "... a string representing an aggregation method available as DataArray/Dataset or *GroupBy methods like DataArray.mean, DataArray.min or DataArray.max. Alternatively, you can pass a callable accepted by DataArray/Dataset.reduce. Alternatively, you can pass a tuple in a format (name, func) where name is used as a coordinate and func is either known string as above or a callable, or (name, func, {kwargs}), if you need to pass additional keyword arguments [[Xvec documentation](https://xvec.readthedocs.io/en/latest/zonal_stats.html)]."


In [ ]:
%%time 

aggregated_custom = ds.xvec.zonal_stats(
    shp.geometry,
    x_coords="x",
    y_coords="y",
    stats=[
        "mean",
        "sum",
        # ("quantile", "quantile", dict(q=[0.1, 0.2, 0.3])),  # commented out this as  the "quantile" aggregation method is not compatible with the current chunking of the dataset along the y dimension. We may need to combine all chunks along the `y` using ds.chunk({"y": -1. 
        ("numpymean", numpy.nanmean),
        numpy.nanstd,
    ],
    name="basin_geometry",    
    index=True,
    method="iterate",
    all_touched=True,
    n_jobs=-1
)

In [ ]:
aggregated_custom